## Qlearning 
it is model free.  
it is off policy means learn from hit and trial  
use bellman equation

In [1]:
import numpy as np
import os
import gym
import random

In [2]:
env = gym.make("FrozenLake-v1",map_name="4x4", is_slippery=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env.reset()

0

In [4]:
#possible actions in this case are 4
print("\n---Action Space____\n")
print("Action space shape:",env.action_space.n)
print('Action space sample:',env.action_space.sample()) #take a random action)


---Action Space____

Action space shape: 4
Action space sample: 2


The action space (the set of possible actions the agent can take) is discrete with 4 actions available 🎮:

0. GO LEFT  
1. GO DOWN  
2. GO RIGHT
3. GO UP

## create and initialize the Q table

In [5]:
state_space = env.observation_space.n
action_space = env.action_space.n
print("There are ", state_space, " possible states") #locations where it can move
print("There are ", action_space, " possible actions")#how it move above,below,left,right

There are  16  possible states
There are  4  possible actions


In [6]:
#states in no of rows and actions in no of columns
def initialize_Q_table(state_space,action_space):
    Qtable=np.zeros((state_space,action_space))
    return Qtable

In [7]:
Qtable_froznlake = initialize_Q_table(state_space,action_space)

In [8]:
Qtable_froznlake

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [9]:
Qtable_froznlake.shape

(16, 4)

1. epsilon greedy policy (exploitation: analyze on which action has max value)
2. With probability ɛ: we do exploration (trying random action).

In [10]:
def epsilon_greedy_policy(Qtable,state,epsilon):
    random_num=random.uniform(0,1)
    if random_num>epsilon:
        action = np.argmax(Qtable[state])

    else:
         action = env.action_space.sample()
    return action

In [11]:
def greedy_policy(Qtable,state):
    action = np.argmax(Qtable[state])
    return action

In [12]:
# Training parameters
n_training_episodes = 10000  # Total training episodes
learning_rate = 0.7          # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate
eval_seed = []               # The evaluation seed of the environment

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05            # Minimum exploration probability
decay_rate = 0.0005            # Exponential decay rate for exploration prob

In [13]:
#initially epsilon vale 1 here itis exploration is 1 b/c it's start explore env
#inc epsilon inc exploration
#dec epsion inc exploitation

### create the training loop method

In [14]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in range(n_training_episodes):
      
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) # Reduce epsilon (because we need less and less exploration)
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # repeat
    for step in range(max_steps):
        
      # Choose the action At using epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, done, info = env.step(action) #Takes the chosen action in the environment

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    #updates the Q-value for the state-action pair using the Q-learning formula
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If done, finish the episode
      if done:
        break

      # Our state is the new state
      state = new_state
  
  return Qtable


        

In [15]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_froznlake)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [16]:
Qtable_frozenlake

array([[0.73509189, 0.77378094, 0.77378094, 0.73509189],
       [0.73509189, 0.        , 0.81450625, 0.77378094],
       [0.77378094, 0.857375  , 0.77378094, 0.81450625],
       [0.81450625, 0.        , 0.77378094, 0.77378094],
       [0.77378094, 0.81450625, 0.        , 0.73509189],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.        , 0.81450625],
       [0.        , 0.        , 0.        , 0.        ],
       [0.81450625, 0.        , 0.857375  , 0.77378094],
       [0.81450625, 0.9025    , 0.9025    , 0.        ],
       [0.857375  , 0.95      , 0.        , 0.857375  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.95      , 0.857375  ],
       [0.9025    , 0.95      , 1.        , 0.9025    ],
       [0.        , 0.        , 0.        , 0.        ]])

In [17]:
import numpy as np

def evaluate_and_plot_with_std(env, Qtable, n_eval_episodes, max_steps):
    rewards_per_episode = []

    for episode in range(n_eval_episodes):
        state = env.reset()
        done = False
        episode_reward = 0

        for step in range(max_steps):
            action = np.argmax(Qtable[state])  # Greedy policy
            new_state, reward, done, info = env.step(action)
            episode_reward += reward

            if done:
                break

            state = new_state

        rewards_per_episode.append(episode_reward)

    # Calculate average reward
    average_reward = np.mean(rewards_per_episode)

    # Calculate standard deviation of rewards
    std_reward = np.std(rewards_per_episode)

    return average_reward, std_reward

In [18]:
# Evaluate our Agent
#mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
mean_reward, std_reward = evaluate_and_plot_with_std(env, Qtable_frozenlake, n_eval_episodes, max_steps)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

Mean_reward=1.00 +/- 0.00


## 8*8 map and slippery

In [19]:
env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True)

In [20]:
env.reset()

0

In [21]:
state_space = env.observation_space.n
action_space= env.action_space.n
print(state_space)
print(action_space)

64
4


In [22]:
def initialize_Q_table(state_space,action_space):
    Qtable=np.zeros((state_space,action_space))
    
    return Qtable

In [23]:
frozen_lake = initialize_Q_table(state_space,action_space)

In [24]:
frozen_lake.shape

(64, 4)

In [25]:
frozen_lake

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [26]:
def epsilon_greedy_policy(Qtable,state,epsilon):
    random_number=random.uniform(0,1)
    if random_number>epsilon:
        action=np.argmax(Qtable[state])
    else:
        action= env.action_space.sample()

    return action

In [27]:
def greedy_policy(Qtable,state):
    action=np.argmax(Qtable[state])
    return action

In [28]:
# Training parameters
n_training_episodes = 10000  # Total training episodes
learning_rate = 0.7          # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate # This is the discounting rate, which determines the importance of future rewards. A value close to 1 means future rewards are highly considered, while a value close to 0 means the agent prioritizes immediate rewards.
eval_seed = []               # The evaluation seed of the environment

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05            # Minimum exploration probability
decay_rate = 0.0005 

In [29]:
# def train(n_training_episodes, min_epsilon, max_epsilon,env, max_steps, decay_rate, Qtable):
#     for episode in range(n_training_episodes):
#         epsilon = min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay_rate*episode)
#         state = env.reset()
#         step=0
#         done=False

#         for step in range(max_steps):
#             action = epsilon_greedy_policy(Qtable, state, epsilon)
#             new_state, reward,info, done = env.step(action) 

#             Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            
#             if done:
#                 break
#             state = new_state
    
#     return Qtable
             

In [30]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in range(n_training_episodes):
      
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) # Reduce epsilon (because we need less and less exploration)
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # repeat
    for step in range(max_steps):
        
      # Choose the action At using epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, done, info = env.step(action) #Takes the chosen action in the environment

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    #updates the Q-value for the state-action pair using the Q-learning formula
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If done, finish the episode
      if done:
        break

      # Our state is the new state
      state = new_state
  
  return Qtable

In [31]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, frozen_lake)

In [32]:
Qtable_frozenlake

array([[2.46521893e-02, 2.48433408e-02, 2.43808542e-02, 2.47896911e-02],
       [2.49022311e-02, 2.54111284e-02, 2.49276291e-02, 2.77446643e-02],
       [2.82401146e-02, 2.97429720e-02, 2.92122215e-02, 3.68443255e-02],
       [3.62072803e-02, 3.34016563e-02, 3.76991249e-02, 3.63463059e-02],
       [4.07959454e-02, 4.50186301e-02, 4.43589567e-02, 5.94803063e-02],
       [5.13615629e-02, 5.31473320e-02, 7.11135959e-02, 5.51672395e-02],
       [6.99849054e-02, 9.55264142e-02, 7.16857604e-02, 6.88548410e-02],
       [7.61131944e-02, 7.07445550e-02, 7.48687068e-02, 7.66611590e-02],
       [2.40778655e-02, 2.52811844e-02, 2.52331294e-02, 2.54220116e-02],
       [2.54707639e-02, 3.18815861e-02, 2.52425337e-02, 2.50434137e-02],
       [1.71010533e-02, 2.77253189e-02, 2.73228794e-02, 4.16815848e-02],
       [1.36898252e-02, 6.76951521e-03, 9.84470885e-03, 4.15951178e-02],
       [4.25436436e-02, 1.99704954e-02, 3.55466599e-02, 3.72383600e-02],
       [3.07147442e-02, 4.43059124e-02, 5.45192976e

In [33]:
import numpy as np

def evaluate_agent(env, Qtable, n_eval_episodes, max_steps):
    rewards_per_episode = []

    for episode in range(n_eval_episodes):
        state = env.reset()
        done = False
        episode_reward = 0

        for step in range(max_steps):
            action = np.argmax(Qtable[state])  # Greedy policy
            new_state, reward, done, info = env.step(action)
            episode_reward += reward

            if done:
                break

            state = new_state

        rewards_per_episode.append(episode_reward)

    # Calculate average reward
    average_reward = np.mean(rewards_per_episode)

    # Calculate standard deviation of rewards
    std_reward = np.std(rewards_per_episode)

    return average_reward, std_reward

In [34]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, Qtable_frozenlake, n_eval_episodes, max_steps)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

Mean_reward=0.02 +/- 0.14


### Taxi